In [11]:
import numpy as np
import pandas as pd

import tensorflow as tf

In [12]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [13]:
#Optimizers

learning_rate = 0.005
epochs = 10
batch_size = 200

#Training data placeholders

x = tf.placeholder(tf.float32, [None, 784])

#Output data placeholder - 10 digits

y = tf.placeholder(tf.float32, [None, 10])

In [14]:
#Declare weights for connectors from i/p to hl1

W1 = tf.Variable(tf.random_normal([784, 300], stddev = 0.03), name = 'W1')
b1 = tf.Variable(tf.random_normal([300], stddev = 0.03), name = "b1")

#Connectors from hl1 to o/p

W2 = tf.Variable(tf.random_normal([300, 10], stddev = 0.03), name = 'W2')
b2 = tf.Variable(tf.random_normal([10], stddev = 0.03, name = 'b2'))

In [15]:
#Calculate the output of the hidden layer

hidden_out = tf.add(tf.matmul(x, W1), b1)       #W1*x + b1
hidden_out = tf.nn.relu(hidden_out)             #relu activation

#dropout activations
#tf.nn.dropout(pkeep) #probability of 50-75% = pkeep

In [16]:
#Hidden layer output - softmax output layer

y_ = tf.nn.softmax(tf.add(tf.matmul(hidden_out,W2), b2))

In [17]:
#to keep the y_ value clipped in (0,1)

y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)

#Cost function to optimize

cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y)*tf.log(1 - y_clipped), axis = 1))

In [18]:
#Optimizer

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cross_entropy)

In [19]:
#Initialisation

init_op = tf.global_variables_initializer()

#Define accuracy

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
#Training

with tf.Session() as sess:
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels)/ batch_size)
    writer = tf.summary.FileWriter("output", sess.graph)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size = batch_size)
            _, c = sess.run([optimizer, cross_entropy],
                           feed_dict = {x: batch_x, y: batch_y})
            avg_cost += c/ total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy*100, feed_dict = {x: mnist.test.images, y: mnist.test.labels}),"%")
    writer.close()

Epoch: 1 cost = 0.426
Epoch: 2 cost = 0.170
Epoch: 3 cost = 0.118
Epoch: 4 cost = 0.092
Epoch: 5 cost = 0.073
Epoch: 6 cost = 0.057
